In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
통합파일_피클 = pd.read_pickle('./data2/통합파일.pkl')

#### <font color='red'>[통합파일_피클 데이터 프레임 인덱스 처리]</font>

In [ ]:
통합파일_피클[통합파일_피클.index == 1]

In [ ]:
통합파일_피클.reset_index(drop = True, inplace=True)
통합파일_피클[통합파일_피클.index == 1]

---

#### <font color='red'>[지번주소에서 '동' 정보 추출]</font>

In [ ]:
pd.DataFrame(통합파일_피클['발생주소(지번)'][:10])

In [ ]:
def 동추출(주소):
    try:      
        주소_동 = re.findall('[가-힣]+[0-9.]*동', 주소)[-1] # 여러개 리스트 값중 마지막 인텍스 값 추출
        주소_동2 = re.findall('[가-힣]+', 주소_동)[0] # 첫 인덱스 값 추출
        주소_동2 =  re.sub('동|본', '', 주소_동2)
#         return 주소_동2
        if 주소_동2 in ['면목', '망우', '상봉', '묵', '신내', '중화']:
            return 주소_동2
        else:
            return '관외'
    except:
        return '기타'

In [ ]:
동추출(통합파일_피클['발생주소(지번)'][0])

In [ ]:
통합파일_피클['주소_동'] = 통합파일_피클['발생주소(지번)'].apply(동추출)

In [ ]:
pd.unique(통합파일_피클.주소_동)

In [ ]:
통합파일_피클.주소_동.value_counts()[:10]

---

#### <font color='red'>Plot내 한글처리를 위해 Matplotlib의 Font를 한글폰트로 지정</font>

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 12

---

#### <font color='red'>동별 코드별 빈도 분석</font>

In [ ]:
plt.figure(figsize=(12, 7)) 
sns.countplot(x='주소_동', hue='코드', data=통합파일_피클, palette='hls', hue_order = ['C0', 'C1', 'C2', 'C3', 'C5', 'C1선'])
plt.grid()
plt.show()

In [ ]:
동별코드별_피봇 = 통합파일_피클.pivot_table('접수번호', index='주소_동', columns='코드', aggfunc='count')
동별코드별_피봇

In [ ]:
동별코드별_피봇.columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C1선']
동별코드별_피봇

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(동별코드별_피봇, annot=True, fmt='.0f', cmap='RdYlGn_r', linewidths=.5, ax=ax)

---

#### <font color='red'>동별 사건종별 빈도 분석</font>

In [ ]:
pd.unique(통합파일_피클.사건종별)

In [ ]:
통합파일_피클.사건종별.value_counts()

In [ ]:
동별사건종별접수_피봇 = 통합파일_피클.pivot_table('접수번호', index='주소_동', columns='사건종별', aggfunc='count')
동별사건종별접수_피봇

In [ ]:
f, ax = plt.subplots(figsize=(20, 6))
sns.heatmap(동별사건종별접수_피봇, annot=True, fmt='.0f', cmap='RdYlGn_r', linewidths=.5, ax=ax)

---

#### <font color='red'>월별 접수건수 추이 분석</font>

In [ ]:
통합파일_피클.info()
통합파일_피클.접수일자 = pd.to_datetime(통합파일_피클.접수일자)

In [ ]:
통합파일_피클.info()

In [ ]:
통합파일_피클.접수일자

In [ ]:
통합파일_피클.접수일자.dt.year

In [ ]:
통합파일_피클.접수일자.dt.month

In [ ]:
통합파일_피클.접수일자.dt.day

In [ ]:
월별 = pd.DataFrame(통합파일_피클.접수일자.dt.month.value_counts())
월별

In [ ]:
월별.reset_index(drop=False, inplace=True)
월별

In [ ]:
월별.columns = ['월', '건수']
월별

In [ ]:
월별.sort_values('월')

In [ ]:
plt.figure(figsize=(7, 7)) 
sns.barplot(x='월', y = '건수', data=월별.sort_values('월'), palette='hls')
plt.grid()
plt.show()

---

#### <font color='red'>일별 접수건수 분석</font>

In [ ]:
일별 = pd.DataFrame(통합파일_피클.접수일자.value_counts())
일별

In [ ]:
일별.reset_index(drop=False, inplace=True)
일별

In [ ]:
일별.columns = ['일', '건수']
일별.일 = 일별.일.astype('str')
일별

In [ ]:
plt.figure(figsize=(30, 15)) 
sns.set(font_scale = 2, font="Malgun Gothic")
sns.barplot(x='일', y = '건수', data=일별.sort_values('일').iloc[:50,], palette='hls')
plt.xticks(rotation=90)
plt.grid()
plt.show()

In [ ]:
sns.set(font="Malgun Gothic", font_scale = 1) 

---

#### <font color='red'>요일별 접수 추이 분석</font>

In [ ]:
통합파일_피클['요일'] = 통합파일_피클.접수일자.dt.weekday

In [ ]:
pd.unique(통합파일_피클.요일)

In [ ]:
통합파일_피클.요일 = 통합파일_피클.요일.map({0:'월요일', 1:'화요일', 2:'수요일', 3:'목요일', 
                                           4:'금요일', 5:'토요일', 6:'일요일'})

In [ ]:
pd.unique(통합파일_피클.요일)

In [ ]:
요일별코드별_피봇 = 통합파일_피클.pivot_table('접수번호', index='요일', columns='코드', aggfunc='count')
요일별코드별_피봇

In [ ]:
요일별코드별_피봇.columns = ['C0', 'C1', 'C2', 'C3', 'C5', 'C1선']
요일별코드별_피봇

In [ ]:
요일별코드별_피봇 = 요일별코드별_피봇.reindex(index = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']) 
요일별코드별_피봇

In [ ]:
plt.figure(figsize=(12, 7)) 
sns.countplot(x=통합파일_피클.요일, hue='코드', data=통합파일_피클, palette='hls', 
              order = ['월요일', '화요일', '수요일', '목요일', '금요일','토요일','일요일'],
              hue_order = ['C0', 'C1', 'C2', 'C3', 'C5', 'C1선'])
plt.grid()
plt.show()

In [ ]:
주소별요일별_피봇 = 통합파일_피클.pivot_table('접수번호', index=['주소_동', '요일'], columns='코드', aggfunc='count')
주소별요일별_피봇

In [ ]:
주소별요일별_주요사건 = 주소별요일별_피봇.reset_index(drop=False)
주소별요일별_주요사건

In [ ]:
관내주소별요일별_주요사건 = 주소별요일별_주요사건[~주소별요일별_주요사건.주소_동.isin(['관외', '기타'])]
관내주소별요일별_주요사건

In [ ]:
관내주소별요일별_주요사건 = 관내주소별요일별_주요사건.iloc[:, :4]
관내주소별요일별_주요사건

In [ ]:
동목록 = pd.unique(관내주소별요일별_주요사건.주소_동)
동목록

In [ ]:
코드값 = 관내주소별요일별_주요사건.C1
최고값 = 코드값.max()

plt.figure(figsize=(20,6))
for i, 동 in enumerate(동목록):  
    plt.subplot(2,3,i+1)
    데이터 = 관내주소별요일별_주요사건[관내주소별요일별_주요사건.주소_동 == 동]
    g = sns.barplot(x=코드값, y= '요일', data=데이터, palette='hls', orient='h')
    g.set(title=동)
    g.set_xlim(0, 최고값)
plt.tight_layout()
plt.show()

---

#### <font color='red'>접수시간대별 분석</font>

In [ ]:
통합파일_피클.info()

In [ ]:
통합파일_피클.접수시간 = pd.to_datetime(통합파일_피클.접수시간, format='%H:%M:%S')
통합파일_피클.info()

In [ ]:
통합파일_피클.접수시간

In [ ]:
통합파일_피클.접수시간.dt.hour

In [ ]:
통합파일_피클['접수시간대'] = 통합파일_피클.접수시간.dt.hour

In [ ]:
통합파일_피클.접수시간대.value_counts()

In [ ]:
plt.figure(figsize=(12, 7)) 
sns.countplot(x='접수시간대',hue='코드', data=통합파일_피클, palette='hls', hue_order = ['C0', 'C1', 'C2', 'C3', 'C5', 'C1선'])
plt.grid()
plt.show()

---

#### <font color='red'>관할관서별 출동소요시간 분석</font>

In [ ]:
통합파일_피클.출동소요시간

In [ ]:
통합파일_피클[통합파일_피클.출동소요시간.notnull() & ~통합파일_피클.출동소요시간.str.contains('0일', na=False)]

In [ ]:
import numpy as np
def 출동소요시간변환(시간):
    if not pd.api.types.is_float(시간):
        시간 = re.sub('0일 ', '', 시간)
    return 시간

In [ ]:
통합파일_피클.출동소요시간 = 통합파일_피클.출동소요시간.apply(출동소요시간변환)
통합파일_피클.출동소요시간

In [ ]:
통합파일_피클.info()

In [ ]:
통합파일_피클.출동소요시간 = pd.to_timedelta(통합파일_피클.출동소요시간, errors='coerce')
통합파일_피클.출동소요시간

In [ ]:
통합파일_피클.출동소요시간.mean()

In [ ]:
통합파일_피클.groupby(['관할관서', '코드'])['출동소요시간']

In [ ]:
통합파일_피클_출동소요시간 = 통합파일_피클[통합파일_피클.출동소요시간.notnull()]

In [ ]:
관할관서별출동소요시간 = pd.DataFrame(통합파일_피클_출동소요시간.groupby(['관할관서', '코드'])['출동소요시간'].mean(numeric_only=False))
관할관서별출동소요시간

In [ ]:
관할관서별출동소요시간.reset_index(drop=False, inplace=True)
관할관서별출동소요시간

In [ ]:
관할관서별출동소요시간 = 관할관서별출동소요시간[관할관서별출동소요시간.코드.isin(['C0', 'C1'])]
관할관서별출동소요시간

In [ ]:
관할관서별출동소요시간['분'] = round(관할관서별출동소요시간.출동소요시간.astype('timedelta64[s]')/60,1)
관할관서별출동소요시간

In [ ]:
관할관서별출동소요시간.drop('출동소요시간', axis=1, inplace=True)
관할관서별출동소요시간

In [ ]:
g = sns.catplot(x='관할관서', kind='bar', y='분', hue='코드', data=관할관서별출동소요시간, palette='hls', hue_order=['C0', 'C1'])
g.set_xticklabels(rotation=45, horizontalalignment='right')
plt.grid()
plt.show()

---

#### <font color='red'>관할관서별 평균 출동소요시간 초과 분석</font>

In [ ]:
통합파일_피클['출동소요시간평균초과여부'] = 통합파일_피클.출동소요시간 > 통합파일_피클.출동소요시간.mean()

In [ ]:
통합파일_피클.출동소요시간평균초과여부

In [ ]:
출동소요시간초과 = 통합파일_피클[통합파일_피클.출동소요시간평균초과여부 & 통합파일_피클.코드.isin(['C0', 'C1'])]
출동소요시간초과

In [ ]:
출동소요시간초과.관할관서.value_counts()

---

#### <font color='red'>접수종결 소요시간 분석</font>

In [ ]:
통합파일_피클['접수일자시간'] = 통합파일_피클.접수일자.astype('str') + ' ' + 통합파일_피클.접수시간.dt.time.astype('str')

In [ ]:
통합파일_피클.접수일자시간

In [ ]:
통합파일_피클.접수일자시간 = pd.to_datetime(통합파일_피클.접수일자시간)

In [ ]:
통합파일_피클.종결시간 = pd.to_datetime(통합파일_피클.종결시간)

In [ ]:
통합파일_피클.종결시간 - 통합파일_피클.접수일자시간

In [ ]:
통합파일_피클['접수종결소요시간'] = round((통합파일_피클.종결시간 - 통합파일_피클.접수일자시간).astype('timedelta64[s]')/60,1)

In [ ]:
통합파일_피클.접수종결소요시간

In [ ]:
plt.figure(figsize=(6,6))
sns.distplot(통합파일_피클.접수종결소요시간)
plt.tight_layout()
plt.show()

In [ ]:
관할관서별접수종결소요시간 = pd.DataFrame(통합파일_피클.groupby(['관할관서', '코드'])['접수종결소요시간'].mean(numeric_only=False))
관할관서별접수종결소요시간

In [ ]:
관할관서별접수종결소요시간.reset_index(drop=False, inplace=True)
관할관서별접수종결소요시간

In [ ]:
관할관서별접수종결소요시간 = round(관할관서별접수종결소요시간, 1)
관할관서별접수종결소요시간

In [ ]:
관할관서별접수종결소요시간 = 관할관서별접수종결소요시간[관할관서별접수종결소요시간.코드.isin(['C0', 'C1'])]
관할관서별접수종결소요시간

In [ ]:
g = sns.catplot(x='관할관서', kind='bar', y='접수종결소요시간', hue='코드',
                data=관할관서별접수종결소요시간, palette='hls',
                hue_order = ['C0', 'C1'])
g.set_xticklabels(rotation=45, horizontalalignment='right')
plt.grid()
plt.show()

---

#### <font color='red'>종결보고자수와 사건처리 소요시간 상관 분석</font>

In [ ]:
list(pd.unique(통합파일_피클.종결보고자))

In [ ]:
re.findall('[가-힣]{2,3}', '노현우,양윤민')

In [ ]:
def 종결보고자_정리(종결보고자):
    종결보고자 = str(종결보고자)
    re.sub('경장|경위','', 종결보고자)
    return re.findall('[가-힣]{2,3}', 종결보고자)

In [ ]:
종결보고자_정리('서미숙,김승훈')

In [ ]:
종결보고자_정리('김진환김선영')

In [ ]:
통합파일_피클.종결보고자 = 통합파일_피클.종결보고자.apply(종결보고자_정리)

In [ ]:
list(통합파일_피클.종결보고자)

In [ ]:
통합파일_피클['종결보고자수'] = 통합파일_피클.종결보고자.apply(lambda li: len(li))

In [ ]:
list(통합파일_피클.종결보고자수)

In [ ]:
통합파일_피클.도착시간

In [ ]:
통합파일_피클.종결시간

In [ ]:
통합파일_도착종결 = 통합파일_피클.loc[통합파일_피클.코드.isin(['C0', 'C1']),['도착시간', '종결시간', '종결보고자수']]
통합파일_도착종결

In [ ]:
통합파일_도착종결.dropna(inplace = True)
통합파일_도착종결.info()

In [ ]:
통합파일_도착종결 = 통합파일_도착종결[통합파일_도착종결.종결보고자수 != 0]
통합파일_도착종결.info()

In [ ]:
통합파일_도착종결.도착시간 = pd.to_datetime(통합파일_도착종결.도착시간)
통합파일_도착종결.info()

In [ ]:
통합파일_도착종결['사건처리시간'] = 통합파일_도착종결.종결시간 - 통합파일_도착종결.도착시간

In [ ]:
통합파일_도착종결.사건처리시간 = round(통합파일_도착종결.사건처리시간.astype('timedelta64[s]')/60,1)

In [ ]:
통합파일_도착종결.사건처리시간

In [ ]:
통합파일_도착종결_상관계수 = 통합파일_도착종결.corr()

In [ ]:
plt.figure(figsize=(6,6))

sns.heatmap(통합파일_도착종결_상관계수, annot=True, cmap='Blues')
plt.show()

---

#### <font color='red'>종결보고자별 분석</font>

In [ ]:
통합파일_피클.종결보고자

In [ ]:
통합파일_피클.pivot_table('종결보고자수', index='관할관서', columns='코드', aggfunc='mean')

In [ ]:
통합파일_피클.종결보고자

In [ ]:
관할관서별종결보고자 = 통합파일_피클.loc[:,['관할관서', '종결보고자']]
관할관서별종결보고자

In [ ]:
관할관서별종결보고자.종결보고자 = 관할관서별종결보고자.종결보고자.apply(lambda li: ','.join(li))
관할관서별종결보고자.종결보고자

In [ ]:
관할관서별종결보고자2 = 관할관서별종결보고자.종결보고자.str.split(',').apply(pd.Series)
관할관서별종결보고자2

In [ ]:
관할관서별종결보고자2.index = 관할관서별종결보고자.set_index('관할관서').index
관할관서별종결보고자2 = 관할관서별종결보고자2.stack().reset_index('관할관서')
관할관서별종결보고자2

In [ ]:
관할관서별종결보고자2.columns = ['관할관서', '종결보고자']
관할관서별종결보고자2

In [ ]:
관할관서별종결보고자2 = 관할관서별종결보고자2[관할관서별종결보고자2.종결보고자!='']

In [ ]:
개인별종결보고실적 = pd.DataFrame(관할관서별종결보고자2.종결보고자.value_counts())
개인별종결보고실적

In [ ]:
개인별종결보고실적.reset_index(drop = False, inplace = True)
개인별종결보고실적

In [ ]:
개인별종결보고실적.columns = ['종결보고자', '실적']
개인별종결보고실적

In [ ]:
plt.figure(figsize=(6,6))
sns.distplot(개인별종결보고실적.실적)
plt.tight_layout()
plt.show()

In [ ]:
관할관서소속 = 관할관서별종결보고자2.groupby('관할관서').agg(lambda x: x.tolist()).reset_index()

In [ ]:
관할관서소속

In [ ]:
관할관서소속['소속종결보고자수'] = 관할관서소속.종결보고자.apply(lambda li: len(li))

In [ ]:
관할관서소속

In [ ]:
관할관서소속['종결보고자'] = 관할관서소속.종결보고자.apply(lambda li: set(li))

In [ ]:
관할관서소속

In [ ]:
관할관서소속['소속종결보고자수'] = 관할관서소속.종결보고자.apply(lambda li: len(li))

In [ ]:
관할관서소속